# Prompt chaining and Sequencing

As AI applications become more sophisticated, there's often a need to break down complex tasks into smaller, manageable steps. Prompt chaining and sequencing allow us to guide language models through a series of interrelated prompts, enabling more structured and controlled outputs. This approach is particularly useful for tasks that require multiple stages of processing or decision-making.

## Key Components

1. **Basic Prompt Chaining**: Connecting the output of one prompt to the input of another.
2. **Sequential Prompting**: Creating a logical flow of prompts to guide the AI through a multi-step process.
3. **Dynamic Prompt Generation**: Using the output of one prompt to dynamically generate the next prompt.
4. **Error Handling and Validation**: Implementing checks and balances within the prompt chain.

In [ ]:
! pip install langchain langchain-google-genai

In [3]:
import os
import re
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ['GOOGLE_API_KEY']=''
llm= ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# Basic Prompt Chaining

In [4]:
# Define prompt template
story_prompt=PromptTemplate(
    input_varibles=['genre'],
    template="""
        Write a short {genre} story in 3-4 sentences.
      """
)

summary_prompt=PromptTemplate(
    input_variables=['story'],
    template='Summarize the following story in one sentence :\n {story}'
)


# Chain the prompt
def story_chain(genre):
    """Generate a story and its summary based on a given genre.

    Args:
        genre (str): The genre of the story to generate.

    Returns:
        tuple: A tuple containing the generated story and its summary.
    """
    story=(story_prompt | llm).invoke({'genre':genre}).content
    summary=(summary_prompt | llm).invoke({'story':story}).content
    return story,summary

# Test the chain
genre='Science Friction'
story,summary=story_chain(genre)
print(f"Story: {story}\n\n summary:{summary}")


Story: The shimmering portal spat out Dr. Aris Thorne, covered in iridescent slime and smelling faintly of ozone.  His experimental wormhole generator had worked, but the destination wasn't the Andromeda galaxy; it was a fungal planet teeming with sentient, bioluminescent mushrooms who demanded tribute in the form of perfectly ripe mangoes.  Aris sighed; paperwork was going to be a nightmare.

 summary:After a mishap with his wormhole generator, Dr. Thorne found himself on a fungal planet populated by sentient mushrooms who demanded mangoes as tribute.


# Sequential Prompting

In [5]:
# Define prompt templates for each step
theme_prompt = PromptTemplate(
    input_variables=["text"],
    template="Identify the main theme of the following text:\n{text}"
)

tone_prompt = PromptTemplate(
    input_variables=["text"],
    template="Describe the overall tone of the following text:\n{text}"
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],
    template="Given the following text with the main theme '{theme}' and tone '{tone}', what are the key takeaways?\n{text}"
)

def analyze_text(text):
    """Perform a multi-step analysis of a given text.

    Args:
        text (str): The text to analyze.

    Returns:
        dict: A dictionary containing the theme, tone, and key takeaways of the text.
    """
    theme = (theme_prompt | llm).invoke({"text": text}).content
    tone = (tone_prompt | llm).invoke({"text": text}).content
    takeaways = (takeaway_prompt | llm).invoke({"text": text, "theme": theme, "tone": tone}).content
    return {"theme": theme, "tone": tone, "takeaways": takeaways}

# Test the sequential prompting
sample_text = "The rapid advancement of artificial intelligence has sparked both excitement and concern among experts. While AI promises to revolutionize industries and improve our daily lives, it also raises ethical questions about privacy, job displacement, and the potential for misuse. As we stand on the brink of this technological revolution, it's crucial that we approach AI development with caution and foresight, ensuring that its benefits are maximized while its risks are minimized."

analysis = analyze_text(sample_text)
for key, value in analysis.items():
    print(f"{key.capitalize()}: {value}\n")

Theme: The main theme is the **ethical considerations and responsible development of artificial intelligence**.  The text highlights both the potential benefits and the significant risks associated with AI, emphasizing the need for careful planning and mitigation of negative consequences.

Tone: The overall tone of the text is **cautiously optimistic**.  It acknowledges the immense potential of AI ("revolutionize industries," "improve our daily lives") while simultaneously highlighting the serious risks and ethical concerns ("ethical questions," "job displacement," "potential for misuse").  The call for "caution and foresight" underscores this balanced and measured perspective.

Takeaways: The key takeaways from the text are:

* **AI's dual nature:** AI offers immense potential benefits (revolutionizing industries, improving daily lives), but also presents significant risks (privacy violations, job displacement, misuse).

* **Ethical considerations are paramount:**  The development and

# Dynamic Prompting

In [6]:
# Define prompt templates
answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely:\n{question}"
)

follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Based on the question '{question}' and the answer '{answer}', generate a relevant follow-up question."
)

def dynamic_qa(initial_question, num_follow_ups=3):
    """Conduct a dynamic Q&A session with follow-up questions.

    Args:
        initial_question (str): The initial question to start the Q&A session.
        num_follow_ups (int): The number of follow-up questions to generate.

    Returns:
        list: A list of dictionaries containing questions and answers.
    """
    qa_chain = []
    current_question = initial_question

    for _ in range(num_follow_ups + 1):  # +1 for the initial question
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        qa_chain.append({"question": current_question, "answer": answer})

        if _ < num_follow_ups:  # Generate follow-up for all but the last iteration
            current_question = (follow_up_prompt | llm).invoke({"question": current_question, "answer": answer}).content

    return qa_chain

# Test the dynamic Q&A system
initial_question = "What are the potential applications of quantum computing?"
qa_session = dynamic_qa(initial_question)

for i, qa in enumerate(qa_session):
    print(f"Q{i+1}: {qa['question']}")
    print(f"A{i+1}: {qa['answer']}\n")

Q1: What are the potential applications of quantum computing?
A1: Drug discovery, materials science, financial modeling, cryptography, optimization problems, and artificial intelligence.

Q2: What are the biggest challenges currently hindering the widespread adoption and practical application of quantum computing in these fields?
A2: High cost, qubit instability, error correction limitations, and lack of readily available algorithms and skilled workforce.

Q3: Given the challenges of high cost, qubit instability, error correction limitations, and lack of skilled workforce, what are the most promising short-term and long-term strategies for overcoming these obstacles and accelerating the adoption of quantum computing?
A3: **Short-term:** Focus on hybrid quantum-classical algorithms, improved qubit coherence, and developing accessible quantum computing platforms & training programs.

**Long-term:**  Invest in fault-tolerant quantum computing architectures, scalable fabrication techniques

# Error Handeling and Validation

In [7]:
# Define prompt templates
generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a 4-digit number related to the topic: {topic}. Respond with ONLY the number, no additional text."
)

validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],
    template="Is the number {number} truly related to the topic '{topic}'? Answer with 'Yes' or 'No' and explain why."
)

def extract_number(text):
    """Extract a 4-digit number from the given text.

    Args:
        text (str): The text to extract the number from.

    Returns:
        str or None: The extracted 4-digit number, or None if no valid number is found.
    """
    match = re.search(r'\b\d{4}\b', text)
    return match.group() if match else None

def robust_number_generation(topic, max_attempts=3):
    """Generate a topic-related number with validation and error handling.

    Args:
        topic (str): The topic to generate a number for.
        max_attempts (int): Maximum number of generation attempts.

    Returns:
        str: A validated 4-digit number or an error message.
    """
    for attempt in range(max_attempts):
        try:
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            number = extract_number(response)

            if not number:
                raise ValueError(f"Failed to extract a 4-digit number from the response: {response}")

            # Validate the relevance
            validation = (validate_prompt | llm).invoke({"number": number, "topic": topic}).content
            if validation.lower().startswith("yes"):
                return number
            else:
                print(f"Attempt {attempt + 1}: Number {number} was not validated. Reason: {validation}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")

    return "Failed to generate a valid number after multiple attempts."

# Test the robust number generation
topic = "World War II"
result = robust_number_generation(topic)
print(f"Final result for topic '{topic}': {result}")

Final result for topic 'World War II': 1945
